In [60]:
# Dependencies
import requests
import time
import csv
import pandas as pd
import datetime as dt
import calendar

In [63]:
#importing the list of areas and zips
zips = pd.read_csv("Resources/zips_ca.csv")
zips.loc[zips["area"]=="bakersfield"]
locations = zips["area"].unique()
locations

array(['bakersfield', 'chico', 'fresno', 'goldcountry', 'hanford',
       'humboldt', 'imperial', 'inlandempire', 'losangeles', 'mendocino',
       'merced', 'modesto', 'monterey', 'orangecounty', 'palmsprings',
       'redding', 'sacramento', 'sandiego', 'santabarbara', 'santamaria',
       'sfbay', 'siskiyou', 'slo', 'stockton', 'susanville', 'ventura',
       'visalia', 'yubasutter'], dtype=object)

In [5]:
###creating a dictionary of the areas and their zips

area_zips = {}

for area in zips["area"].unique():
    local_zip = zips.loc[zips["area"]==area]
    local_zip = local_zip["zip"]
    
    for zip_ in local_zip:
        if area not in area_zips.keys():
            area_zips[area]=[zip_]
        else:
            area_zips[area].append(zip_)
    
area_zips["bakersfield"]

[93203,
 93205,
 93206,
 93207,
 93215,
 93220,
 93222,
 93224,
 93225,
 93226,
 93238,
 93240,
 93241,
 93243,
 93250,
 93251,
 93255,
 93260,
 93263,
 93268,
 93276,
 93283,
 93285,
 93287,
 93301,
 93304,
 93305,
 93306,
 93307,
 93308,
 93309,
 93311,
 93312,
 93313,
 93314,
 93501,
 93505,
 93518,
 93519,
 93527,
 93528,
 93531,
 93554,
 93555,
 93558,
 93561,
 93562,
 93592]

In [291]:
area_zips["redding"]

[96001,
 96002,
 96003,
 96007,
 96008,
 96011,
 96013,
 96016,
 96019,
 96022,
 96024,
 96028,
 96029,
 96033,
 96047,
 96048,
 96051,
 96052,
 96059,
 96062,
 96065,
 96069,
 96073,
 96074,
 96075,
 96076,
 96080,
 96084,
 96087,
 96088,
 96091,
 96093,
 96096]

In [6]:
###creating a dictionary of the categories and their ids from ebay

cat_ids = {"antiques": [20081],
           "arts_crafts": [14339],
           "atv_utv_sno": [43962, 66466],
           "auto_parts": [6030],
           "aviation": [63676],
           "baby_kids": [2984, 3082, 171146],
           "beauty_health": [26395],
           "bike_parts": [7294],
           "bikes": [177831],
           "boat_parts": [26443],
           "boats": [26429],
           "books": [267],
           "cars_trucks": [6001],
           "cds_dvds_vhs": [176984, 617, 309],
           "cell_phones": [15032],
           "clothes_acc": [15724, 1059, 14324, 3034, 93427, 169291, 4251, 4250],
           "computer_parts": [175673],
           "computers": [58058],
           "furniture": [3197],
           "heavy_equip": [257887],
           "jewelry": [4196, 10968, 48579, 91427, 10290],
           "motorcycle_parts": [10063],
           "motorcycles": [6024],
           "music_intr": [619],
           "photo_video": [625],
           "rvs_camp": [50054],
           "sporting": [888],
           "tools": [631],
           "toys_games": [220],
           "trailers": [66468],
           "video_gaming": [1249]
          }

for k, v in cat_ids.items():
    print(k)
    for id in v:
        print(id)

antiques
20081
arts_crafts
14339
atv_utv_sno
43962
66466
auto_parts
6030
aviation
63676
baby_kids
2984
3082
171146
beauty_health
26395
bike_parts
7294
bikes
177831
boat_parts
26443
boats
26429
books
267
cars_trucks
6001
cds_dvds_vhs
176984
617
309
cell_phones
15032
clothes_acc
15724
1059
14324
3034
93427
169291
4251
4250
computer_parts
175673
computers
58058
furniture
3197
heavy_equip
257887
jewelry
4196
10968
48579
91427
10290
motorcycle_parts
10063
motorcycles
6024
music_intr
619
photo_video
625
rvs_camp
50054
sporting
888
tools
631
toys_games
220
trailers
66468
video_gaming
1249


In [9]:
###generating the dataframe of urls to run a json query from

#first key is Kyle's, second is Sean's
api_key = ["KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a", "SeanTibb-SeanTibb-PRD-b7da08f45-1e07c1f5"]

url_df = []
max_api_call_per_day_count = 0
api_alternate = 0
loc_url_count = 0

url_df = pd.DataFrame(columns=["location", "zip", "category", "cat_id", "url"])

for loc, zips in area_zips.items():
    loc_url_count = 0
    print("Location: "+ loc + " started.")  #if errors, to check where
    for zip_ in zips:
        for cat, ids in cat_ids.items():
            id_urls = {}
            for id in ids:
                loc_url_count +=1
                max_api_call_per_day_count +=1
                if max_api_call_per_day_count > 5000: #max calls per key per day is 5000
                    api_alternate += 1 #used to alternate between api keys.  we will modulus to swap later
                    max_api_call_per_day_count = 0 #resets if we're reaching 5000 calls
                
                query_url = "https://svcs.ebay.com/services/search/FindingService/v1?\
OPERATION-NAME=findItemsByCategory&\
SERVICE-VERSION=1.0.0&\
SECURITY-APPNAME="+api_key[api_alternate%2]+"&\
buyerPostalCode="+str(zip_)+"&\
itemFilter.name=MaxDistance&\
itemFilter.value=5&\
RESPONSE-DATA-FORMAT=JSON&\
REST-PAYLOAD&\
categoryId="+str(id)+"&\
paginationInput.entriesPerPage=1000"
            
                temp_df = {"location":loc, 
                           "zip":zip_, 
                           "category":cat, 
                           "cat_id":id, 
                           "url":query_url}
                
                url_df = url_df.append(temp_df, ignore_index=True)
    print("Location: "+ loc + " finished, "+str(loc_url_count)+" total urls generated.")  #if errors, to check where
    print("-------------------------------------")

url_df.head()

Location: bakersfield started.
Location: bakersfield finished, 2256 total urls generated.
-------------------------------------
Location: chico started.
Location: chico finished, 1222 total urls generated.
-------------------------------------
Location: fresno started.
Location: fresno finished, 1786 total urls generated.
-------------------------------------
Location: goldcountry started.
Location: goldcountry finished, 2961 total urls generated.
-------------------------------------
Location: hanford started.
Location: hanford finished, 846 total urls generated.
-------------------------------------
Location: humboldt started.
Location: humboldt finished, 2068 total urls generated.
-------------------------------------
Location: imperial started.
Location: imperial finished, 846 total urls generated.
-------------------------------------
Location: inlandempire started.
Location: inlandempire finished, 1175 total urls generated.
-------------------------------------
Location: losangel

,location,zip,category,cat_id,url
0,bakersfield,93203,antiques,20081,https://svcs.ebay.com/services/search/FindingS...
1,bakersfield,93203,arts_crafts,14339,https://svcs.ebay.com/services/search/FindingS...
2,bakersfield,93203,atv_utv_sno,43962,https://svcs.ebay.com/services/search/FindingS...
3,bakersfield,93203,atv_utv_sno,66466,https://svcs.ebay.com/services/search/FindingS...
4,bakersfield,93203,auto_parts,6030,https://svcs.ebay.com/services/search/FindingS...


In [40]:
#url_df.to_csv("Resources/full_list_of_urls.csv", index=False)

In [76]:
imported_df = []
filtered_url_df = []

imported_df = pd.read_csv("Resources/full_list_of_urls.csv")
bakersfield_url_df = imported_df.loc[imported_df["location"]=="bakersfield"]
sacramento_url_df = imported_df.loc[imported_df["location"]=="sacramento"]
redding_url_df = imported_df.loc[imported_df["location"]=="redding"]
sandiego_url_df = imported_df.loc[imported_df["location"]=="sandiego"]

In [73]:
len(redding_url_df), len(bakersfield_url_df), len(sacramento_url_df), len(sandiego_url_df)

(1551, 2256, 3525, 4606)

In [81]:
filtered_url_df = []
filtered_url_df = pd.DataFrame
filtered_url_df = pd.concat([bakersfield_url_df, redding_url_df, sacramento_url_df, sandiego_url_df])
filtered_url_df = filtered_url_df.reset_index(drop=True)
filtered_url_df.to_csv("Resources/filtered_list_of_urls.csv", index=False)

In [82]:
filtered_url_df

,location,zip,category,cat_id,url
0,bakersfield,93203,antiques,20081,https://svcs.ebay.com/services/search/FindingS...
1,bakersfield,93203,arts_crafts,14339,https://svcs.ebay.com/services/search/FindingS...
2,bakersfield,93203,atv_utv_sno,43962,https://svcs.ebay.com/services/search/FindingS...
3,bakersfield,93203,atv_utv_sno,66466,https://svcs.ebay.com/services/search/FindingS...
4,bakersfield,93203,auto_parts,6030,https://svcs.ebay.com/services/search/FindingS...
5,bakersfield,93203,aviation,63676,https://svcs.ebay.com/services/search/FindingS...
6,bakersfield,93203,baby_kids,2984,https://svcs.ebay.com/services/search/FindingS...
7,bakersfield,93203,baby_kids,3082,https://svcs.ebay.com/services/search/FindingS...
8,bakersfield,93203,baby_kids,171146,https://svcs.ebay.com/services/search/FindingS...
9,bakersfield,93203,beauty_health,26395,https://svcs.ebay.com/services/search/FindingS...


# code from here on out will need to be babysat

In [286]:
###change for each day.  "first_day...", "second_day...", etc
###be sure to save into new csv
todays_requests = pd.read_csv("Resources/first_day_of_calls_urls.csv")
todays_requests = import_requests.iloc[0:8526]
todays_requests

,location,zip,category,cat_id,url
0,bakersfield,93203,antiques,20081,https://svcs.ebay.com/services/search/FindingS...
1,bakersfield,93203,arts_crafts,14339,https://svcs.ebay.com/services/search/FindingS...
2,bakersfield,93203,atv_utv_sno,43962,https://svcs.ebay.com/services/search/FindingS...
3,bakersfield,93203,atv_utv_sno,66466,https://svcs.ebay.com/services/search/FindingS...
4,bakersfield,93203,auto_parts,6030,https://svcs.ebay.com/services/search/FindingS...
5,bakersfield,93203,aviation,63676,https://svcs.ebay.com/services/search/FindingS...
6,bakersfield,93203,baby_kids,2984,https://svcs.ebay.com/services/search/FindingS...
7,bakersfield,93203,baby_kids,3082,https://svcs.ebay.com/services/search/FindingS...
8,bakersfield,93203,baby_kids,171146,https://svcs.ebay.com/services/search/FindingS...
9,bakersfield,93203,beauty_health,26395,https://svcs.ebay.com/services/search/FindingS...


In [280]:
todays_requests = todays_requests[0:10]

In [ ]:
###
json_list = []
json_count = 0
for index, row in todays_requests.iterrows():
    json_count+= 1
    this_request = requests.get(row["url"]).json()
    json_list.append([row["url"], this_request])
    print("Pulling request "+str(json_count))
        
json_list[0]

In [295]:
json_list[10]#["findItemsByCategoryResponse"][0]["searchResult"][0]["item"]#[1]["shippingInfo"][0]["shippingServiceCost"]

['https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=93203&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=7294&paginationInput.entriesPerPage=1000',
 {'findItemsByCategoryResponse': [{'ack': ['Success'],
    'version': ['1.13.0'],
    'timestamp': ['2019-07-04T07:03:01.550Z'],
    'searchResult': [{'@count': '10',
      'item': [{'itemId': ['153552509987'],
        'title': ['Powerlite expert bmx frame cruiser early 2000'],
        'globalId': ['EBAY-US'],
        'primaryCategory': [{'categoryId': ['22679'],
          'categoryName': ['Bicycle Frames']}],
        'galleryURL': ['http://thumbs4.ebaystatic.com/m/mkKB2amRqY_mng1FwzmZOXQ/140.jpg'],
        'viewItemURL': ['http://www.ebay.com/itm/Powerlite-expert-bmx-frame-cruiser-early-2000-/153552509987'],
        'paymentMethod': ['PayPal'],
        'au

In [98]:
json_list_temp_df = pd.DataFrame(json_list)

# json_list_temp_df.to_csv("Resources/temp_json_file.csv", index=False)

In [267]:
json_list[1][0]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=92007&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=15724&paginationInput.entriesPerPage=1000'

In [ ]:
##testing how to pull json elements out
json_df_one = []
json_df_two = []
json_df_three = []
json_df_four = []

json_df_one = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])
json_df_two = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])
json_df_three = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])
json_df_four = pd.DataFrame(columns=["url", "results_count", "result_title", "result_currency",\
                               "result_list_price", "result_ship_price", "free_shipping"])

#
# if (json_list.index(this_thing)//2500) == 0:
#     #append to df1
# elif (json_list.index(this_thing)//2500) == 1:
#     #append to df2
# elif (json_list.index(this_thing)//2500) == 2:
#     #append to df3
# else:
#     #append to df4

for this_thing in json_list:
    results_count = 0
    result_title = ""
    result_currency = ""
    result_list_price = 0
    result_ship_price = 0
    free_shipping = ""
    try:
        this_search = this_thing[1]["findItemsByCategoryResponse"][0]["searchResult"][0]
    ###number of results
        results_count = this_search["@count"]
        if int(results_count) > 0:
            for the_item in this_search["item"]:

            ###title
                result_title = this_search["item"][this_search["item"].index(the_item)]['title']
            ###currency
                result_currency = this_search["item"][this_search["item"].index(the_item)]["sellingStatus"][0]["currentPrice"][0]["@currencyId"]
            ###price
                result_list_price = this_search["item"][this_search["item"].index(the_item)]["sellingStatus"][0]["currentPrice"][0]["__value__"]
            ###shipping price
                try:
                    result_ship_price = this_search["item"][this_search["item"].index(the_item)]["shippingInfo"][0]["shippingServiceCost"][0]["__value__"]
                ###free shipping
                    free_shipping = this_search["item"][this_search["item"].index(the_item)]["shippingInfo"][0]["shippingType"][0] == 'Free'

                except:
                    result_ship_price = "unknown"
                    free_shipping = False

                temp_df = {"url": this_thing[0],
                      "results_count":results_count,
                      "result_title": result_title, 
                      "result_currency": result_currency,
                      "result_list_price": result_list_price,
                      "result_ship_price": result_ship_price,
                      "free_shipping": free_shipping}
                if (json_list.index(this_thing)//2500) == 0:
                    #append to df1
                    json_df_one = json_df_one.append(temp_df, ignore_index=True)
                elif (json_list.index(this_thing)//2500) == 1:
                    #append to df2
                    json_df_two = json_df_two.append(temp_df, ignore_index=True)
                elif (json_list.index(this_thing)//2500) == 2:
                    #append to df3
                    json_df_three = json_df_three.append(temp_df, ignore_index=True)
                else:
                    #append to df4
                    json_df_four = json_df_four.append(temp_df, ignore_index=True)

        else:
            temp_df = {"url": this_thing[0],
                      "results_count":results_count,
                      "result_title": result_title, 
                      "result_currency": result_currency,
                      "result_list_price": result_list_price,
                      "result_ship_price": result_ship_price,
                      "free_shipping": free_shipping}

            if (json_list.index(this_thing)//2500) == 0:
                #append to df1
                json_df_one = json_df_one.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 1:
                #append to df2
                json_df_two = json_df_two.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 2:
                #append to df3
                json_df_three = json_df_three.append(temp_df, ignore_index=True)
            else:
                #append to df4
                json_df_four = json_df_four.append(temp_df, ignore_index=True)  

        print(json_list.index(this_thing))
    except:
            temp_df = {"url": this_thing[0],
                      "results_count": results_count,
                      "result_title": "ERROR", 
                      "result_currency": result_currency,
                      "result_list_price": result_list_price,
                      "result_ship_price": result_ship_price,
                      "free_shipping": free_shipping}
            if (json_list.index(this_thing)//2500) == 0:
                #append to df1
                json_df_one = json_df_one.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 1:
                #append to df2
                json_df_two = json_df_two.append(temp_df, ignore_index=True)
            elif (json_list.index(this_thing)//2500) == 2:
                #append to df3
                json_df_three = json_df_three.append(temp_df, ignore_index=True)
            else:
                #append to df4
                json_df_four = json_df_four.append(temp_df, ignore_index=True)  

In [289]:
json_df = pd.concat([json_df_one, json_df_two, json_df_three, json_df_four], ignore_index=True)
# len(json_df_one)+len(json_df_two)
json_df["url"][200]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=93203&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=10968&paginationInput.entriesPerPage=1000'

In [270]:
json_df["url"][500]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=KyleDavi-DAProjec-PRD-5721a6b08-1eb0707a&buyerPostalCode=92007&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=93427&paginationInput.entriesPerPage=1000'

In [290]:
###change file name and 
json_df.to_csv("Resources/zero_day_json_parsed_FIXED.csv", index=False)

In [ ]:
### place to assign that day's pulls
day_one_json_pulls = json_list

In [194]:
json_df = pd.concat([json_df_one, json_df_two, json_df_three, json_df_four], ignore_index=True)
# len(json_df_one)+len(json_df_two)

In [88]:
test_list = [[1,2], [3,4]]
test_list

[[1, 2], [3, 4]]

In [66]:
test_list.append(1)

In [67]:
test_list

[1]

In [93]:
for item in test_list:
    print(test_list.index(item))

0
1


In [180]:
(4999//2500)

1

In [169]:
json_list[1][1]

{'findItemsByCategoryResponse': [{'ack': ['Success'],
   'version': ['1.13.0'],
   'timestamp': ['2019-06-29T21:57:10.658Z'],
   'searchResult': [{'@count': '13',
     'item': [{'itemId': ['362507715486'],
       'title': ['Handmade Paper Beads Glazed Recycled Loose Painted Lot 50 Ct - PPM01'],
       'globalId': ['EBAY-US'],
       'primaryCategory': [{'categoryId': ['146290'],
         'categoryName': ['Beads']}],
       'galleryURL': ['http://thumbs3.ebaystatic.com/m/mE7msvkGZbOybSuQFrEoG3w/140.jpg'],
       'viewItemURL': ['http://www.ebay.com/itm/Handmade-Paper-Beads-Glazed-Recycled-Loose-Painted-Lot-50-Ct-PPM01-/362507715486'],
       'paymentMethod': ['PayPal'],
       'autoPay': ['false'],
       'postalCode': ['93203'],
       'location': ['Arvin,CA,USA'],
       'country': ['US'],
       'shippingInfo': [{'shippingServiceCost': [{'@currencyId': 'USD',
           '__value__': '0.0'}],
         'shippingType': ['Free'],
         'shipToLocations': ['Worldwide'],
         'exped

In [240]:
this_thing[0]

'https://svcs.ebay.com/services/search/FindingService/v1?OPERATION-NAME=findItemsByCategory&SERVICE-VERSION=1.0.0&SECURITY-APPNAME=SeanTibb-SeanTibb-PRD-b7da08f45-1e07c1f5&buyerPostalCode=93624&itemFilter.name=MaxDistance&itemFilter.value=5&RESPONSE-DATA-FORMAT=JSON&REST-PAYLOAD&categoryId=3082&paginationInput.entriesPerPage=1000'